In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\harsh\\OneDrive\\Desktop\\Job\\ResumeProjects\\SterileBrowsing\\SterileBrowsingMedicalImgs'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    params_image_size: list
    params_batch_size: int
    params_classes: int
    params_loss: str
    params_metrics: list
    params_epochs: int


In [6]:
from cnnImageClassification.constants import *
from cnnImageClassification.utils.common import read_yaml, create_directories

In [7]:
class ConfigManager:
    def __init__(
            self, 
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root]) 
    
    def get_prep_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])

        return PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            params_classes=self.params.CLASSES,
            params_loss=self.params.LOSS,
            params_metrics=self.params.METRICS,
            params_epochs=self.params.EPOCHS  # Ensure this line is added
        )


In [8]:
import os
import urllib.request as request
import zipfile as ZipFile
import tensorflow as tf

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

In [10]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config


    
    def define_base_model(self):
        self.model = Sequential([
                Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(self.config.params_image_size[0], self.config.params_image_size[1], self.config.params_image_size[2])),
                BatchNormalization(),
                MaxPooling2D(pool_size=(2, 2)),
                Dropout(0.25),
                
                Conv2D(64, kernel_size=(3, 3), activation='relu'),
                BatchNormalization(),
                MaxPooling2D(pool_size=(2, 2)),
                Dropout(0.25),
                
                Flatten(),
                Dense(128, activation='relu'),  # Fully connected layer
                Dropout(0.5),
                
            ])

        self.save_model(path=self.config.base_model_path, model=self.model)


    
    @staticmethod
    def _prepare_full_model(model, classes, loss, epochs, metrics,batch_size):
      
        prediction = Dense(units=classes, activation="softmax")(model.output)
        full_model = tf.keras.Model(inputs=model.input, outputs=prediction)

        # Compile the model
        full_model.compile(
            optimizer=Adam(),
            loss=loss,
            metrics=metrics
        )

        full_model.summary()
        return full_model
    

    def create_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            loss=self.config.params_loss,
            epochs=self.config.params_epochs,
            metrics=self.config.params_metrics,
            batch_size=self.config.params_batch_size
        )

        self.save_model(path=self.config.base_model_path, model=self.full_model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [11]:
try:
    config = ConfigManager()
    prepare_base_model_config = config.get_prep_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.define_base_model()
    prepare_base_model.create_base_model()
except Exception as e:
    raise e

[2024-10-28 15:18:49,183: INFO: common{c:\users\harsh\onedrive\desktop\job\resumeprojects\sterilebrowsing\sterilebrowsingmedicalimgs\src\cnnImageClassification\utils\common.py: Reading yaml file from config\config.yaml loaded suucessfully]
[2024-10-28 15:18:49,191: INFO: common{c:\users\harsh\onedrive\desktop\job\resumeprojects\sterilebrowsing\sterilebrowsingmedicalimgs\src\cnnImageClassification\utils\common.py: Reading yaml file from params.yaml loaded suucessfully]
[2024-10-28 15:18:49,191: INFO: common{c:\users\harsh\onedrive\desktop\job\resumeprojects\sterilebrowsing\sterilebrowsingmedicalimgs\src\cnnImageClassification\utils\common.py: Directory created at artifacts]
[2024-10-28 15:18:49,199: INFO: common{c:\users\harsh\onedrive\desktop\job\resumeprojects\sterilebrowsing\sterilebrowsingmedicalimgs\src\cnnImageClassification\utils\common.py: Directory created at artifacts/prepare_base_model]
[2024-10-28 15:18:50,406: WARNING: saving_utils{c:\Users\harsh\anaconda3\envs\py310TFGPU\l